<a href="https://colab.research.google.com/github/IndigoClay/cudaCompileDeviceQuery/blob/main/cudaCompileDeviceQuery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
Thu Mar 31 12:38:38 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |            

In [2]:
!git clone https://github.com/IndigoClay/cudaCompileDeviceQuery.git

Cloning into 'cudaCompileDeviceQuery'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 22 (delta 6), reused 16 (delta 3), pack-reused 0
Unpacking objects: 100% (22/22), done.


In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

In [3]:
!pip install git+https://github.com/IndigoClay/nvcc4jupyter.git

  Cloning https://github.com/IndigoClay/nvcc4jupyter.git to /tmp/pip-req-build-77xjwnhl
  Running command git clone -q https://github.com/IndigoClay/nvcc4jupyter.git /tmp/pip-req-build-77xjwnhl
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4286 sha256=22c801c020c2e4ce0372b29798840b483c739ceacafc28f769514cd0817c3e28
  Stored in directory: /tmp/pip-ephem-wheel-cache-4pyg8qsp/wheels/c4/52/63/f7029f491c28a55cf4cb96d4abbcb3460592100edbd8560c5e
Successfully built NVCCPlugin


In [4]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cu
#include <stdio.h>

#define NUM_BLOCKS 16
#define BLOCK_WIDTH 1

__global__ void hello()
{
    printf("Hello world! I'm a thread in block %d\n", blockIdx.x);
}


int main(int argc,char **argv)
{
    // launch the kernel
    hello<<<NUM_BLOCKS, BLOCK_WIDTH>>>();

    // force the printf()s to flush
    cudaDeviceSynchronize();
    
    printf("That's all!\n");

    return 0;
}

In [9]:
!cd /content/cudaCompileDeviceQuery && pwd && make

/content/cudaCompileDeviceQuery
make: Nothing to be done for 'all'.


In [ ]:
!./cudaCompileDeviceQuery/deviceQuery

In [57]:
!pwd

/content


In [58]:
%%cu
#include <cuda_runtime.h>
#include <iostream>

//demonstating the usage of global memory

using namespace std;
__global__ void use_global_memory_GPU(float *array)
{
    array[threadIdx.x] = 3.14f * (float) threadIdx.x;
}

int main(int argc, char **argv)
{
    float h_arr[128];
    float *d_arr;
 
    cudaMalloc((void **)&d_arr, sizeof(float)*128);
    cudaMemcpy((void *)d_arr, (void *)h_arr, sizeof(float)*128, cudaMemcpyHostToDevice);
 
    use_global_memory_GPU<<<1, 128>>>(d_arr);
    cudaMemcpy((void *)h_arr, (void *)d_arr, sizeof(float)*128, cudaMemcpyDeviceToHost);
    cudaFree(d_arr);
    for (auto const& i : h_arr) {
      cout << i << "  ";
    }
    cout << h_arr[2];
    //system("ls");
}

0  3.14  6.28  9.42  12.56  15.7  18.84  21.98  25.12  28.26  31.4  34.54  37.68  40.82  43.96  47.1  50.24  53.38  56.52  59.66  62.8  65.94  69.08  72.22  75.36  78.5  81.64  84.78  87.92  91.06  94.2  97.34  100.48  103.62  106.76  109.9  113.04  116.18  119.32  122.46  125.6  128.74  131.88  135.02  138.16  141.3  144.44  147.58  150.72  153.86  157  160.14  163.28  166.42  169.56  172.7  175.84  178.98  182.12  185.26  188.4  191.54  194.68  197.82  200.96  204.1  207.24  210.38  213.52  216.66  219.8  222.94  226.08  229.22  232.36  235.5  238.64  241.78  244.92  248.06  251.2  254.34  257.48  260.62  263.76  266.9  270.04  273.18  276.32  279.46  282.6  285.74  288.88  292.02  295.16  298.3  301.44  304.58  307.72  310.86  314  317.14  320.28  323.42  326.56  329.7  332.84  335.98  339.12  342.26  345.4  348.54  351.68  354.82  357.96  361.1  364.24  367.38  370.52  373.66  376.8  379.94  383.08  386.22  389.36  392.5  395.64  398.78  6.28


In [42]:
%%cu
#include <cuda_runtime.h>
#include <iostream>
#include "/content/cudaCompileDeviceQuery/GpuTimer.h"
using namespace std;

//demonstating the usage of global and shared memory and the start of 2 kernels
__global__ void use_global_memory_GPU(float *array)
{
    array[threadIdx.x] = 1.0f * (float) threadIdx.x;
}


__global__ void use_shared_memory_GPU(float *array)
{
   
    int i, index = threadIdx.x;
    float average, sum = 0.0f;
 
    __shared__ float sh_arr[128];
    
    // copy data from global array to shared array
    sh_arr[index] = array[index];
 
    __syncthreads();
 
    // sum up values until own index
    for (i=0; i<=index; i++) {
        sum += sh_arr[i];
    }
    // the average = sum/NumElements = sum/index
    average = sum / (index+1.0f);
    //array[index] = sum;
    if(array[index] > average) { array[index] = average;}
}

int main(int argc, char argv)
{
    GpuTimer timer;
    float h_arr[128];
    float *d_arr;
 
    cudaMalloc((void **)&d_arr, sizeof(float)*128);
    cudaMemcpy((void *)d_arr, (void *)h_arr, sizeof(float)*128, cudaMemcpyHostToDevice);
 
    use_global_memory_GPU<<<1, 128>>>(d_arr);
    cudaMemcpy((void *)h_arr, (void *)d_arr, sizeof(float)*128, cudaMemcpyDeviceToHost);
     for (auto const& i : h_arr) {
      cout << i << "  ";
    }
    cout << endl;
    cudaMemcpy((void *)d_arr, (void *)h_arr, sizeof(float)*128, cudaMemcpyHostToDevice);
    timer.Start();
    use_shared_memory_GPU<<<1, 128>>>(d_arr);
    timer.Stop();
    cudaMemcpy((void *)h_arr, (void *)d_arr, sizeof(float)*128, cudaMemcpyDeviceToHost);
    for (auto const& i : h_arr) {
      cout << i << "  ";
    }
    cout << endl;
    printf("Time elapsed = %g ms", timer.Elapsed());
    cudaFree(d_arr);

   
    //system("ls");
}


0  1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99  100  101  102  103  104  105  106  107  108  109  110  111  112  113  114  115  116  117  118  119  120  121  122  123  124  125  126  127  
0  0.5  1  1.5  2  2.5  3  3.5  4  4.5  5  5.5  6  6.5  7  7.5  8  8.5  9  9.5  10  10.5  11  11.5  12  12.5  13  13.5  14  14.5  15  15.5  16  16.5  17  17.5  18  18.5  19  19.5  20  20.5  21  21.5  22  22.5  23  23.5  24  24.5  25  25.5  26  26.5  27  27.5  28  28.5  29  29.5  30  30.5  31  31.5  32  32.5  33  33.5  34  34.5  35  35.5  36  36.5  37  37.5  38  38.5  39  39.5  40  40.5  41  41.5  42  42.5  43  43.5  44  44.5  45  45.5  46  46.5  47  47.5  48  48.5 

In [55]:
%%cu
#include <stdio.h>
#include "/content/cudaCompileDeviceQuery/GpuTimer.h"

#define NUM_THREADS 1000000
#define ARRAY_SIZE  100

#define BLOCK_WIDTH 1000

void print_array(int *array, int size)
{
    printf("{ ");
    for (int i = 0; i < size; i++)  { printf("%d ", array[i]); }
    printf("}\n");
}

__global__ void increment_naive(int *g)
{
	// which thread is this?
	int i = blockIdx.x * blockDim.x + threadIdx.x; 

	// each thread to increment consecutive elements, wrapping at ARRAY_SIZE
	i = i % ARRAY_SIZE;  
	g[i] = g[i] + 1;
}

__global__ void increment_atomic(int *g)
{
	// which thread is this?
	int i = blockIdx.x * blockDim.x + threadIdx.x; 

	// each thread to increment consecutive elements, wrapping at ARRAY_SIZE
	i = i % ARRAY_SIZE;  
	atomicAdd(& g[i], 1);
}

int main(int argc,char **argv)
{   
    GpuTimer timer;
    printf("%d total threads in %d blocks writing into %d array elements\n",
           NUM_THREADS, NUM_THREADS / BLOCK_WIDTH, ARRAY_SIZE);

    // declare and allocate host memory
    int h_array[ARRAY_SIZE];
    const int ARRAY_BYTES = ARRAY_SIZE * sizeof(int);
 
    // declare, allocate, and zero out GPU memory
    int * d_array;
    cudaMalloc((void **) &d_array, ARRAY_BYTES);
    cudaMemset((void *) d_array, 0, ARRAY_BYTES); 

    // launch the kernel - comment out one of these
    timer.Start();
    
    // Instructions: This program is needed for the next quiz
    // uncomment increment_naive to measure speed and accuracy 
    // of non-atomic increments or uncomment increment_atomic to
    // measure speed and accuracy of  atomic icrements
    increment_naive<<<NUM_THREADS/BLOCK_WIDTH, BLOCK_WIDTH>>>(d_array);
    //increment_atomic<<<NUM_THREADS/BLOCK_WIDTH, BLOCK_WIDTH>>>(d_array);
    timer.Stop();
    
    // copy back the array of sums from GPU and print
    cudaMemcpy(h_array, d_array, ARRAY_BYTES, cudaMemcpyDeviceToHost);
    print_array(h_array, ARRAY_SIZE);
    printf("Time elapsed = %g ms\n", timer.Elapsed());
 
    // free GPU memory allocation and exit
    cudaFree(d_array);
    return 0;
}

1000000 total threads in 1000 blocks writing into 100 array elements
{ 55 55 55 55 56 56 56 56 58 58 58 58 58 58 58 58 57 57 57 57 56 56 56 56 56 56 56 56 59 59 59 59 67 67 67 67 63 63 63 63 63 63 63 63 57 57 57 57 57 57 57 57 58 58 58 58 61 61 61 61 53 53 53 53 52 52 52 52 51 51 51 51 51 51 51 51 47 47 47 47 48 48 48 48 48 48 48 48 52 52 52 52 52 52 52 52 54 54 54 54 }
Time elapsed = 0.707296 ms

